In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Import libraries**

In [ ]:
import pandas as pd
import numpy as np
import keras

from keras.layers import LSTM
from numpy import array
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from numpy import array
from numpy import asarray
from numpy import zeros
import gc


**Reading data**

In [ ]:
train=pd.read_csv("../input/dsfsnvdknb/train_processed.csv")
test=pd.read_csv("../input/testda/test_processed.csv")

 **Extract values for train the model**

In [ ]:
sentences = train['Final'].values
y = train['BROWSE_NODE_ID'].values



**Clear the ram for better performance**

In [ ]:
del train
gc.collect()

**Encoding of classes**

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)


In [ ]:
sentences_train, sentences_test, y_train, y_test = train_test_split(sentences, y, test_size=0.20, random_state=1000)


**Initializing tokenizer**

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences_train)
tokenizer.fit_on_texts(sentences_test)



**Convert token to a sequence**

In [ ]:

X_train = tokenizer.texts_to_sequences(sentences_train)
X_test = tokenizer.texts_to_sequences(sentences_test)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index


In [ ]:
del sentences
gc.collect()

**Defining same length for every list**

In [ ]:
maxlen = 25

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

print(X_train[0, :])


**Model architecture**

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 25, input_length=maxlen))
model.add(LSTM(512, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(9919, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())



In [ ]:
history=model.fit(X_train, y_train,validation_data=(X_test,y_test),epochs=20,batch_size=10000)


**Ploting model accuracy history**

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

**Ploting model loss history**

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

**Model performance**

In [ ]:
print(model.evaluate(X_test,y_test)[1]*100)

**Save the model for future purpose**

In [ ]:
model.save("./Amazon_ML_2021.h5")

**Same process for test data**

In [ ]:
#lets predict on test on test data
test_sen=test['Final'].values

In [ ]:
test_sentest_sen = tokenizer.texts_to_sequences(test_sen)


In [ ]:
test_sentest_sen = pad_sequences(test_sentest_sen, padding='post', maxlen=maxlen)


In [ ]:
test_pred=model.predict(test_sentest_sen)

**Extracting classes**

In [ ]:
predictions=[]
for i in range(len(test_pred[0])):
    predictions.append(test_pred[i].argmax())


**Final Result**

In [ ]:
#we had used label encoder while training so we have to inverse the encoding for real classes
Final=pd.DataFrame({'PRODUCT_ID':test['PRODUCT_ID'],'BROWSE_NODE_ID':label_encoder.inverse_transform(predictions)})

In [ ]:
Final.to_csv("./Final.csv",index=False)